In [ ]:
"""
waste_data.py  ·  Data & Metrics Layer
--------------------------------------
Stores and evaluates rules for the Rule-Based Waste Classifier.
"""

import json, logging
from pathlib import Path
from collections import Counter
from typing import Dict, Tuple, Optional

# ── configuration ────────────────────────────────────────────────────
RULES_FILE   = Path("waste_rules.json")
LOG_FILE     = Path("classifier.log")

logging.basicConfig(filename=LOG_FILE,
                    level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# ── shipped knowledge base ───────────────────────────────────────────
DEFAULT_RULES: Dict[str, Tuple[str, str]] = {
    "banana peel"   : ("Organic",   "Compost"),
    "plastic bottle": ("Plastic",   "Recycle"),
    "battery"       : ("Hazardous", "Dispose at collection point"),
    "glass jar"     : ("Glass",     "Recycle"),
    "egg shell"     : ("Organic",   "Compost"),
    "can"           : ("Metal",     "Recycle"),
    "newspaper"     : ("Paper",     "Recycle"),
    "food wrapper"  : ("Plastic",   "Dispose")
}

SYNONYMS = {
    "tin can":  "can",
    "bottle":   "plastic bottle",
    "wrapper":  "food wrapper",
    "bulb":     "light bulb"
}

# ── persistence helpers ─────────────────────────────────────────────
def _save_rules(rules: Dict[str, Tuple[str, str]]) -> None:
    RULES_FILE.write_text(json.dumps(rules, indent=2, ensure_ascii=False))

def _load_rules() -> Dict[str, Tuple[str, str]]:
    if RULES_FILE.exists():
        return json.loads(RULES_FILE.read_text())
    _save_rules(DEFAULT_RULES)
    return DEFAULT_RULES.copy()

# ── public façade ───────────────────────────────────────────────────
class RuleStore:
    """Tiny wrapper around the JSON rule file + synonyms dict."""

    def __init__(self) -> None:
        self.rules: Dict[str, Tuple[str, str]] = _load_rules()
        self.syns  = SYNONYMS

    # -- CRUD ---------------------------------------------------------
    def add_rule(self, item: str, category: str, action: str) -> None:
        self.rules[item.lower().strip()] = (category, action)
        _save_rules(self.rules)

    def get(self, term: str) -> Optional[Tuple[str, str]]:
        return self.rules.get(term)

    def synonym_for(self, maybe: str) -> str:
        return self.syns.get(maybe, maybe)

    # -- evaluation ---------------------------------------------------
    def evaluate(self, ground_truth: dict[str, Optional[str]], classifier) -> None:
        """Print Acc / Prec / Rec / F1 using an injected `classifier(term)` call."""
        y_true, y_pred = [], []
        for item, truth in ground_truth.items():
            pred, _ = classifier(item)          # returns (cat, action)
            if truth is not None:
                y_true.append(truth)
                y_pred.append(pred or "Unknown")

        labels = sorted(set(y_true + y_pred))
        cm = Counter(zip(y_true, y_pred))

        print("Label        Prec  Rec  F1  Support")
        for lbl in labels:
            tp = cm.get((lbl, lbl), 0)
            fp = sum(cm.get((o, lbl), 0) for o in labels if o != lbl)
            fn = sum(cm.get((lbl, o), 0) for o in labels if o != lbl)
            sup = tp + fn
            prec = tp/(tp+fp) if tp+fp else 0
            rec  = tp/sup      if sup   else 0
            f1   = 2*prec*rec/(prec+rec) if prec+rec else 0
            print(f"{lbl:<12} {prec:5.2f} {rec:5.2f} {f1:5.2f}  {sup:7}")

        acc = sum(cm.get((l,l),0) for l in labels)/len(y_true)*100
        print(f"\nAccuracy: {acc:.1f}% on {len(y_true)} samples\n")